In [1]:
# https://tarunlalwani.com/post/reusing-existing-browser-session-selenium/
# https://medium.com/better-programming/lets-create-an-instagram-bot-to-show-you-the-power-of-selenium-349d7a6744f7
# https://towardsdatascience.com/web-scraping-using-selenium-and-beautifulsoup-99195cd70a58
# https://medium.com/@srujana.rao2/scraping-instagram-with-python-using-selenium-and-beautiful-soup-8b72c186a058
import time
from bs4 import BeautifulSoup
import bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests

%matplotlib inline
import matplotlib.pyplot as plt
import uuid
import pandas as pd

## Open browser to login page

In [2]:
email = 'harvestharvest894@gmail.com'
password = 'earplugs99'

browser = webdriver.Chrome()
browser.get('https://www.instagram.com/accounts/login/ ')

## Login

In [3]:
emailInput = browser.find_elements_by_css_selector('form input')[0]
passwordInput = browser.find_elements_by_css_selector('form input')[1]

emailInput.send_keys(email)
passwordInput.send_keys(password)
passwordInput.send_keys(Keys.ENTER)

time.sleep(2)

## Dismiss notification pop

In [4]:
browser.find_element_by_xpath('//button[text()="Not Now"]').click()

## Search for profile

In [5]:
# search_input = browser.find_element_by_xpath("//input[@placeholder='Search']")

In [6]:
# search_input.send_keys('pizza')
# time.sleep(1)

## Scroll through page and collect source

In [7]:
# more_button = browser.find_element_by_xpath('//button[text()="more"]')

In [8]:
# more_button.click()

In [9]:
# for more_button in browser.find_elements_by_xpath('//button[text()="more"]'):
#     more_button.click()

In [10]:
# browser.get_window_size()['height']

In [11]:
browser.execute_script("window.scrollBy(0, {});".format(browser.get_window_size()['height']))

In [12]:
SCROLL_PAUSE_TIME = 1.0
soups = []
# Get scroll height
# last_height = browser.execute_script("return document.body.scrollHeight")
# last_height = browser.get_window_size()['height']
scroll_position = 0  

for _ in range(10):
#     for more_button in browser.find_elements_by_xpath('//button[text()="more"]'):
#         print('Clicking more')
#         more_button.click()
    time.sleep(SCROLL_PAUSE_TIME)
    
    soups.append(BeautifulSoup(browser.page_source))
    
    # Scroll down to bottom
#     browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     browser.execute_script("window.scrollTo(0, {});".format(browser.get_window_size()['height']))
    browser.execute_script("window.scrollTo(0, {});".format(scroll_position))

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
#     new_height = browser.execute_script("return document.body.scrollHeight")
#     print('New height: {}'.format(new_height))
#     print('New height: {}'.format(last_height))
#     if new_height == last_height:
#         break
    scroll_position += browser.get_window_size()['height']
#     last_height = new_height
#     break

## Grab all posts

In [13]:
# soup = BeautifulSoup(browser.page_source)

# articles = soup.find_all('article')

In [14]:
def extract_from_post_article(article: bs4.element.Tag):
    post_images = [image for image in article.find_all('img') if 'profile picture' not in image.get('alt')]
    post_image = post_images[0]
    alt_text = post_image.get('alt')
    img_insta_url = post_image.get('src')
    
    influencer = article.contents[0].text
    
    caption = article.contents[2].find_all(recursive=False)[2].text
    if caption.startswith(influencer):
        caption = caption[len(influencer):]

    caption = caption.split('View all')[0]
    
    id = uuid.uuid4()
    
    r = requests.get(img_insta_url)
    img_file = "tmp/{}_{}.jpg".format(influencer, id)
    with open(img_file, 'wb') as f:
        f.write(r.content)
        
    return {
        'alt_text': alt_text,
        'img_insta_url': img_insta_url,
        'influencer': influencer,
        'influencer_caption': caption,
        'id': id,
        'img_file': img_file,
    }
    
    
# extract_from_post_article(articles[0])

In [15]:
def process_soup(soup):
    articles = soup.find_all('article')
    
    processed_articles = []
    for article in articles:
        processed_articles.append(
            extract_from_post_article(article)
        )
    return pd.DataFrame(processed_articles)

article_dfs = [process_soup(soup) for soup in soups]

In [16]:
scrapped_df = pd.concat(article_dfs)
scrapped_df.head()

,alt_text,img_insta_url,influencer,influencer_caption,id,img_file
0,,https://scontent-yyz1-1.cdninstagram.com/v/t51...,dogs.therapy,10% OFF Today using Discount Code : DOGLOVE (l...,932df00f-d380-4fc2-9d29-bc5b6415e8d7,tmp/dogs.therapy_932df00f-d380-4fc2-9d29-bc5b6...
1,Image may contain: dog,https://scontent-yyz1-1.cdninstagram.com/v/t51...,puppytoday,,8b3604e8-b294-4fe8-873c-3194a59ac478,tmp/puppytoday_8b3604e8-b294-4fe8-873c-3194a59...
2,Image may contain: dog and outdoor,https://scontent-yyz1-1.cdninstagram.com/v/t51...,puppytoday,,aadb5910-1004-4ec6-953f-90de1e3cab16,tmp/puppytoday_aadb5910-1004-4ec6-953f-90de1e3...
3,"Image may contain: people sitting, dog and text",https://scontent-yyz1-1.cdninstagram.com/v/t51...,dogs.therapy,samesies.😭❤... more,1d71a94e-72ce-4563-a834-8f96ae43de86,tmp/dogs.therapy_1d71a94e-72ce-4563-a834-8f96a...
0,,https://scontent-yyz1-1.cdninstagram.com/v/t51...,dogs.therapy,10% OFF Today using Discount Code : DOGLOVE (l...,76ca0751-ab36-4354-ac2e-c5930ce6981a,tmp/dogs.therapy_76ca0751-ab36-4354-ac2e-c5930...


In [17]:
import base64

def encode_image(f):
    with open(f, "rb") as img_file:
        encoded_image = base64.b64encode(img_file.read())
    return encoded_image

scrapped_df['img_base64'] = scrapped_df['img_file'].apply(encode_image)

In [18]:
def hash_post(x):
    return str(hash(x['influencer'] + x['influencer_caption'] + str(hash(x['img_base64']))))
scrapped_df['id'] = scrapped_df.apply(hash_post, axis=1)

In [19]:

from app.models import Post

In [20]:
from contextlib import contextmanager
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

DATABASE_URL="postgresql://localhost:5432/insta_clone"
engine = create_engine(DATABASE_URL)

Session = sessionmaker(bind=engine)
@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [21]:
scrapped_df.head()

,alt_text,img_insta_url,influencer,influencer_caption,id,img_file,img_base64
0,,https://scontent-yyz1-1.cdninstagram.com/v/t51...,dogs.therapy,10% OFF Today using Discount Code : DOGLOVE (l...,8201613119994951548,tmp/dogs.therapy_932df00f-d380-4fc2-9d29-bc5b6...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/7QB8UGhvdG9zaG9w...
1,Image may contain: dog,https://scontent-yyz1-1.cdninstagram.com/v/t51...,puppytoday,,3037839132033785413,tmp/puppytoday_8b3604e8-b294-4fe8-873c-3194a59...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/7QB8UGhvdG9zaG9w...
2,Image may contain: dog and outdoor,https://scontent-yyz1-1.cdninstagram.com/v/t51...,puppytoday,,-3645183433122581777,tmp/puppytoday_aadb5910-1004-4ec6-953f-90de1e3...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/7QB8UGhvdG9zaG9w...
3,"Image may contain: people sitting, dog and text",https://scontent-yyz1-1.cdninstagram.com/v/t51...,dogs.therapy,samesies.😭❤... more,-9167216749164457050,tmp/dogs.therapy_1d71a94e-72ce-4563-a834-8f96a...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/7QB8UGhvdG9zaG9w...
0,,https://scontent-yyz1-1.cdninstagram.com/v/t51...,dogs.therapy,10% OFF Today using Discount Code : DOGLOVE (l...,8201613119994951548,tmp/dogs.therapy_76ca0751-ab36-4354-ac2e-c5930...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/7QB8UGhvdG9zaG9w...


In [23]:
from shutil import copyfile



def save_post_if_new(post, session):
    print('Id: {}'.format(post['id']))
    existing_matches = session.query(Post).filter_by(id=post['id']).first()
    if not existing_matches:
        print('No existing post found, saving')
        copyfile(post['img_file'], 'data/{}_{}.png'.format(post['influencer'].replace('.', '_'), post['id']))
        post['img_file'] = '{}_{}.png'.format(post['influencer'].replace('.', '_'), post['id'])
        influencer_post = Post(**post[['id', 'influencer', 'img_file', 'influencer_caption', 'alt_text']].to_dict())
        session.add(influencer_post)
        session.commit()
        
with session_scope() as session:
    scrapped_df.apply(save_post_if_new, axis=1, session=session)

Id: 8201613119994951548
No existing post found, saving
Id: 3037839132033785413
No existing post found, saving
Id: -3645183433122581777
No existing post found, saving
Id: -9167216749164457050
No existing post found, saving
Id: 8201613119994951548
Id: 3037839132033785413
Id: -3645183433122581777
Id: 8201613119994951548
Id: 3037839132033785413
Id: -3645183433122581777
Id: -9167216749164457050
Id: 8201613119994951548
Id: 3037839132033785413
Id: -3645183433122581777
Id: -9167216749164457050
Id: 27994054083348994
No existing post found, saving
Id: 3616173502171678554
No existing post found, saving
Id: 8201613119994951548
Id: 3037839132033785413
Id: -3645183433122581777
Id: -9167216749164457050
Id: 27994054083348994
Id: 3616173502171678554
Id: 3037839132033785413
Id: -3645183433122581777
Id: -9167216749164457050
Id: 27994054083348994
Id: 3616173502171678554
Id: 6530405874601374406
No existing post found, saving
Id: 3037839132033785413
Id: -3645183433122581777
Id: -9167216749164457050
Id: 2799